# Desarrollo y datos calculados

### Interpolacion curva de temperaturas
En base a la tabla de pruebas de pendientes por cada $1° C$ hechas a partir de 0.5, 1 y 1.5 lts de carga con la pava voy a interpolar para sacar la curva de pendientes a distintas cargas de agua. Tengo solo los extremos y un punto medio, lo tomo como suficiente para no romper mi pava haciendole tantas pruebas. Uso el polinomio interpolador de Lagrange
![tabla pendientes](tabla_pendientes.png)

In [7]:
#primero tabla de diferencias de temperatura con respecto a carga de agua
mediciones_pendiente = [0.5, 1;
                            1, 0.45;
                            1.5, 0.31];
#Después de cuánto se pasa la temperatura a distintos niveles de agua
mediciones_DT = [
                        0.5, 12;
                        1, 8;
                        1.5, 4];

tabla_pendientes = linspace(0.5, 1.5, 100)';
tabla_pendientes(:, 2) = tabla_pendientes(:, 1);
tabla_pendientes(:, 1) = [1:length(tabla_pendientes(:,1))]';

In [8]:
function L = coef_lagrange(tabla, j, x)
    L=1;
    for k=[1:length(tabla)]
        if(k==j)
            continue
        end
    L *= (x - tabla(k, 1)) / (tabla(j, 1) - tabla(k, 1));
    end
end

function valor = lagrange(tabla, x)
    valor=0;;
    for j=[1:length(tabla)]
        valor += tabla(j, 2) * coef_lagrange(tabla, j, x);
    end
end

In [9]:
for x=[1: length(tabla_pendientes(:, 1))]
    tabla_pendientes(x, 3) = lagrange(mediciones_pendiente, tabla_pendientes(x, 2));
    tabla_pendientes(x, 4) = lagrange(mediciones_DT, tabla_pendientes(x, 2));
end

In [10]:
h=figure;
#x=rand(1,10);
#y=1:10;
plot(tabla_pendientes(:,2),tabla_pendientes(:,3),'r');
saveas(h, 'tabla_pendientes','png');
g=figure;
plot(tabla_pendientes(:,2), tabla_pendientes(:,4), 'b')
saveas(g, 'tabla_dif_t', 'png')

La curva roja es la pendiente de cambio de temperatura con respecto a la carga. La azul, los grados que se pasa de la temperatura seteada con respecto a la carga de agua. Ambos están interpolados, yo solo tengo tres datos.

![tabla_pendientes](tabla_pendientes.png)
![tabla curvas temperatura](tabla_dif_t.png)

In [11]:
tabla_pendientes = tabla_pendientes(:, [3:4]);#No necesito ni posicion ni niveles de agua

In [12]:
save tabla_pendientes

In [13]:
load tabla_pendientes

In [14]:
tabla_pendientes

tabla_pendientes =

    1.0000   12.0000
    0.9848   11.9192
    0.9698   11.8384
    0.9550   11.7576
    0.9403   11.6768
    0.9258   11.5960
    0.9115   11.5152
    0.8973   11.4343
    0.8833   11.3535
    0.8695   11.2727
    0.8558   11.1919
    0.8423   11.1111
    0.8290   11.0303
    0.8159   10.9495
    0.8029   10.8687
    0.7900   10.7879
    0.7774   10.7071
    0.7649   10.6263
    0.7526   10.5455
    0.7404   10.4646
    0.7284   10.3838
    0.7166   10.3030
    0.7049   10.2222
    0.6935   10.1414
    0.6821   10.0606
    0.6710    9.9798
    0.6600    9.8990
    0.6492    9.8182
    0.6385    9.7374
    0.6280    9.6566
    0.6177    9.5758
    0.6076    9.4949
    0.5976    9.4141
    0.5878    9.3333
    0.5781    9.2525
    0.5687    9.1717
    0.5593    9.0909
    0.5502    9.0101
    0.5412    8.9293
    0.5324    8.8485
    0.5238    8.7677
    0.5153    8.6869
    0.5070    8.6061
    0.4988    8.5253
    0.4909    8.4444
    0.4831    8.3636
    0.4754    